In [60]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Populating database

In [61]:
ourschema = """
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY NOT NULL,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
)
"""

### SQLITE

In [62]:
# Sqlite is a text or memory based database. Connecte and get a DBAPI2 connection
from sqlite3 import dbapi2 as sq3
import os
PATSTART = "."
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATSTART, dbfile))
    return sqlite_db

In [63]:
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

In [64]:
# use Pandas to read in the data.
dfcand = pd.read_csv('candidates.txt', sep='|')
dfcwci = pd.read_csv('contributors_with_candidate_id.txt',sep='|')
del dfcwci['id']
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16


### Init
Initializing the database

In [65]:
db = init_db('cancont.db', ourschema)

### Populating with Pandas!!

In [66]:
dfcand.to_sql("candidates", db, if_exists="append", index=False)

In [67]:
dfcwci.to_sql("contributors", db, if_exists="append", index=False)

In [68]:
dfcwci

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16
5,Akin,Mike,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,1500,2007-05-18,16
6,Akin,Rebecca,NaN,181 Baywood Lane,NaN,Monticello,AR,71655,500,2007-05-18,16
7,Aldridge,Brittni,NaN,"808 Capitol Square Place, SW",NaN,Washington,DC,20024,250,2007-06-06,16
8,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1000,2007-06-11,16
9,Allen,John D.,NaN,1052 Cannon Mill Drive,NaN,North Augusta,SC,29860,1300,2007-06-29,16


In [69]:
sql = """
SELECT * FROM candidates;
"""
c=db.cursor().execute(sql)
c.fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

In [70]:
rem = """
DELETE FROM candidates;
"""
c = db.cursor().execute(rem)
db.commit()
c.fetchall()    

[]

#### Populate with SQL INSERT

In [71]:
ins = """
INSERT INTO candidates (id, first_name, last_name, middle_name, party) VALUES (?, ?, ?, ?, ?);
"""
with open("candidates.txt") as fd:
    slines = [l.strip().split('|') for l in fd.readlines()]
    for line in slines[1:]:
        theid, first_name, last_name, middle_name, party = line
        print (theid, first_name, last_name, middle_name, party)
        valstoinsert = (int(theid), first_name, last_name, middle_name, party)
        print (ins, valstoinsert)
        db.cursor().execute(ins, valstoinsert)

33 Joseph Biden  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party) VALUES (?, ?, ?, ?, ?);
 (33, 'Joseph', 'Biden', '', 'D')
36 Samuel Brownback  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party) VALUES (?, ?, ?, ?, ?);
 (36, 'Samuel', 'Brownback', '', 'R')
34 Hillary Clinton R. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party) VALUES (?, ?, ?, ?, ?);
 (34, 'Hillary', 'Clinton', 'R.', 'D')
39 Christopher Dodd J. D

INSERT INTO candidates (id, first_name, last_name, middle_name, party) VALUES (?, ?, ?, ?, ?);
 (39, 'Christopher', 'Dodd', 'J.', 'D')
26 John Edwards  D

INSERT INTO candidates (id, first_name, last_name, middle_name, party) VALUES (?, ?, ?, ?, ?);
 (26, 'John', 'Edwards', '', 'D')
22 Rudolph Giuliani  R

INSERT INTO candidates (id, first_name, last_name, middle_name, party) VALUES (?, ?, ?, ?, ?);
 (22, 'Rudolph', 'Giuliani', '', 'R')
24 Mike Gravel  D

INSERT INTO candidates (id, first_name, last_name

In [72]:
def make_query(sql):
    c=db.cursor().execute(sql);
    return c.fetchall()

In [73]:
make_query("SELECT * FROM candidates;")

[(16, 'Mike', 'Huckabee', '', 'R'),
 (20, 'Barack', 'Obama', '', 'D'),
 (22, 'Rudolph', 'Giuliani', '', 'R'),
 (24, 'Mike', 'Gravel', '', 'D'),
 (26, 'John', 'Edwards', '', 'D'),
 (29, 'Bill', 'Richardson', '', 'D'),
 (30, 'Duncan', 'Hunter', '', 'R'),
 (31, 'Dennis', 'Kucinich', '', 'D'),
 (32, 'Ron', 'Paul', '', 'R'),
 (33, 'Joseph', 'Biden', '', 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', '', 'R'),
 (36, 'Samuel', 'Brownback', '', 'R'),
 (37, 'John', 'McCain', '', 'R'),
 (38, 'Tom', 'Tancredo', '', 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

### Query

In [74]:
dfcwci.query("state=='VA' & amount < 400")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [75]:
dfcwci[(dfcwci.state=='VA') & (dfcwci.amount < 400)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [76]:
cont_cols = [e[1] for e in make_query("PRAGMA table_info(contributors);")]
cont_cols

['id',
 'last_name',
 'first_name',
 'middle_name',
 'street_1',
 'street_2',
 'city',
 'state',
 'zip',
 'amount',
 'date',
 'candidate_id']

In [83]:
def make_frame(list_of_tubles, legend=cont_cols):
    framelist = []
    for i, cname in enumerate(legend):
        framelist.append((cname, [e[i] for e in list_of_tubles]))
    return pd.DataFrame.from_items(framelist)

In [84]:
out = make_query("SELECT * FROM contributors where state = 'VA' AND amount < 400;")
print(out)
make_frame(out)

[(28, 'Buckheit', 'Bruce', None, '8904 KAREN DR', None, 'FAIRFAX', 'VA', '220312731', 100, '2007-09-19', 20), (78, 'Ranganath', 'Anoop', None, '2507 Willard Drive', None, 'Charlottesville', 'VA', '22903', -100, '2008-04-21', 32), (89, 'Perreault', 'Louise', None, '503 Brockridge Hunt Drive', None, 'Hampton', 'VA', '23666', -34.08, '2008-04-21', 32), (146, 'ABDELLA', 'THOMAS', 'M.', '4231 MONUMENT WALL WAY #340', None, 'FAIRFAX', 'VA', '220308440', 50, '2007-09-30', 35)]


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
1,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
2,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
3,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [85]:
out = make_query("SELECT * FROM contributors WHERE state IS NULL;")
make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,None,LAGE KAART 77,None,BRASSCHATT,None,2930,-500,2008-11-20,35


In [86]:
dfcwci[dfcwci.state.isnull()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500,2008-11-20,35


In [94]:
out = make_query("SELECT * FROM contributors WHERE state IS NOT NULL;")
make_frame(out).shape

(174, 12)

In [95]:
dfcwci[dfcwci.state.notnull()].shape

(174, 11)

In [98]:
out = make_query("SELECT * FROM contributors WHERE state in ('VA', 'WA');")
make_frame(out).head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16
1,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
2,63,BURKE,SUZANNE,M.,3401 EVANSTON,None,SEATTLE,WA,981038677,-700.00,2008-03-05,22
3,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
4,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32


In [99]:
dfcwci[dfcwci.state.isin(['VA', 'WA'])].head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
62,BURKE,SUZANNE,M.,3401 EVANSTON,NaN,SEATTLE,WA,981038677,-700.00,2008-03-05,22
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32


In [100]:
dfcwci['name'] = dfcwci['last_name'] + ", " + dfcwci['first_name']
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,"Agee, Steven"
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don"
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"


In [101]:
dfcwci.assign(ucname=dfcwci.last_name +":"+dfcwci.first_name).head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name,ucname
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500,2007-06-30,16,"Agee, Steven",Agee:Steven
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don",Ahrens:Don
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don",Ahrens:Don
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don",Ahrens:Don
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles",Akin:Charles


In [102]:
dfcwci[dfcwci.state=='VA']

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16,"Agee, Steven"
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20,"Buckheit, Bruce"
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32,"Ranganath, Anoop"
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32,"Perreault, Louise"
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35,"ABDELLA, THOMAS"


In [105]:
dfcwci.loc[dfcwci.state=='VA', 'name']

0           Agee, Steven
27       Buckheit, Bruce
77      Ranganath, Anoop
88     Perreault, Louise
145      ABDELLA, THOMAS
Name: name, dtype: object

In [112]:
alt = "ALTER TABLE contributors ADD column name;"
db.cursor().execute(alt)

In [107]:
make_query("PRAGMA table_info(contributors);")

[(0, 'id', 'INTEGER', 1, None, 1),
 (1, 'last_name', 'VARCHAR', 0, None, 0),
 (2, 'first_name', 'VARCHAR', 0, None, 0),
 (3, 'middle_name', 'VARCHAR', 0, None, 0),
 (4, 'street_1', 'VARCHAR', 0, None, 0),
 (5, 'street_2', 'VARCHAR', 0, None, 0),
 (6, 'city', 'VARCHAR', 0, None, 0),
 (7, 'state', 'VARCHAR', 0, None, 0),
 (8, 'zip', 'VARCHAR', 0, None, 0),
 (9, 'amount', 'INTEGER', 0, None, 0),
 (10, 'date', 'DATETIME', 0, None, 0),
 (11, 'candidate_id', 'INTEGER', 1, None, 0)]

In [110]:
out = make_query("SELECT id, last_name, first_name from contributors;")
out2 = [(e[1] + ", " + e[2], e[0]) for e in out]
out2

[('Agee, Steven', 1),
 ('Ahrens, Don', 2),
 ('Ahrens, Don', 3),
 ('Ahrens, Don', 4),
 ('Akin, Charles', 5),
 ('Akin, Mike', 6),
 ('Akin, Rebecca', 7),
 ('Aldridge, Brittni', 8),
 ('Allen, John D.', 9),
 ('Allen, John D.', 10),
 ('Allison, John W.', 11),
 ('Allison, Rebecca', 12),
 ('Allison, Rebecca', 13),
 ('Altes, R.D.', 14),
 ('Andres, Dale', 15),
 ('Anthony, John', 16),
 ('Arbogast, Robert', 17),
 ('Arbogast, Robert', 18),
 ('Ardle, William', 19),
 ('Atiq, Omar', 20),
 ('Atiq, Omar', 21),
 ('Baker, David', 22),
 ('Bancroft, David', 23),
 ('Banks, Charles', 24),
 ('Barbee, John', 25),
 ('Buckler, Steve', 26),
 ('Buckler, Steve', 27),
 ('Buckheit, Bruce', 28),
 ('Buckel, Linda', 29),
 ('Buckel, Linda', 30),
 ('Buckel, Linda', 31),
 ('Buck, Thomas', 32),
 ('Buck, Jay', 33),
 ('Buck, Blaine', 34),
 ('Buck, Barbara', 35),
 ('Buck, Barbara', 36),
 ('Buchman, Mark M', 37),
 ('Bucher, Ida', 38),
 ('Buchanek, Elizabeth', 39),
 ('Buchanan, John', 40),
 ('Buchanan, John', 41),
 ('Buchanan, Jo

In [113]:
alt2 = "UPDATE contributors SET name = ? WHERE id = ?;"
for ele in out2:
    db.cursor().execute(alt2, ele)
db.commit()

In [114]:
out = make_query("SELECT * FROM contributors;")
make_frame(out, cont_cols +["name"]).head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,name
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500,2007-06-30,16,"Agee, Steven"
1,2,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,250,2007-05-16,16,"Ahrens, Don"
2,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50,2007-06-18,16,"Ahrens, Don"
3,4,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,100,2007-06-21,16,"Ahrens, Don"
4,5,Akin,Charles,None,10187 Sugar Creek Road,None,Bentonville,AR,72712,100,2007-06-16,16,"Akin, Charles"


### AGGREGATE

In [115]:
dfcwci.describe()

,zip,amount,candidate_id
count,1.750000e+02,175.000000,175.000000
mean,3.780014e+08,3.418114,28.000000
std,3.628278e+08,1028.418999,7.823484
min,2.474000e+03,-2592.000000,16.000000
25%,9.336700e+04,-175.000000,20.000000
50%,3.233313e+08,100.000000,32.000000
75%,7.816946e+08,300.000000,35.000000
max,9.951532e+08,4600.000000,37.000000


In [116]:
dfcwci.amount.max()

4600.0

In [121]:
dfcwci.groupby('state').sum()

,zip,amount,candidate_id
state,,,
AK,2985459621,1210.00,111
AR,864790,14200.00,192
AZ,860011121,120.00,37
CA,14736360720,-5013.73,600
CO,2405477834,-5823.00,111
CT,68901376,2300.00,35
DC,800341853,-1549.91,102
FL,8970626520,-4050.00,803
IA,50266,250.00,16


In [122]:
dfcwci.groupby('state')['amount'].sum()

state
AK     1210.00
AR    14200.00
AZ      120.00
CA    -5013.73
CO    -5823.00
CT     2300.00
DC    -1549.91
FL    -4050.00
IA      250.00
ID     -261.00
IL    -5586.80
KS     -330.00
KY     -200.00
LA     1300.00
MA      -83.00
MD      300.00
ME     2520.00
MI    -1265.00
MN      322.00
MO      100.00
NC      500.00
NH      -24.60
NJ     -817.45
NV      725.00
NY    -6474.50
OH      450.00
OK      800.00
PA    -2146.00
RI      200.00
SC     2400.00
TN      -25.00
TX     1985.24
UT     5050.00
VA      515.92
WA     -500.00
Name: amount, dtype: float64

In [123]:
dfcwci.state.unique()

array(['VA', 'CA', 'AR', 'DC', 'SC', 'IA', 'OH', 'NC', 'UT', 'MO', 'IL',
       'ME', 'FL', 'MD', 'MI', 'CO', 'WA', 'NY', 'TX', 'KY', 'PA', 'TN',
       'MA', 'MN', 'KS', 'NJ', 'NH', 'ID', 'OK', nan, 'NV', 'CT', 'RI',
       'AK', 'LA', 'AZ'], dtype=object)

In [124]:
db.close()